In [1]:
import sys
import os

pwd = %pwd
path = pwd
while True:
    try:
        sys.path.append(path)
        from __init__ import *
    except ModuleNotFoundError:
        sys.path.remove(path)
        path = os.path.dirname(path)
    else:
        break
print("pwd:", pwd)

from CODE.Utils.augmentation import Augmentation
from CODE.Train.defence import Defence

HOME_LOC: /Project/Project/ADMA2024/v0.1
pwd: /Project/Project/ADMA2024/v0.1/CODE/Demo/TEST


In [3]:
_ = list(TRAIN_MODEL_LIST)
pprint(_)
i_model = _[0]
print(i_model)

[<class 'CODE.Train.TS_2_V.main.Classifier_TS2V'>,
 <class 'CODE.Train.resnet.ClassifierResNet18'>,
 <class 'CODE.Train.inception_time.Classifier_INCEPTION'>,
 <class 'CODE.Train.macnn.Classifier_MACNN'>,
 <class 'CODE.Train.lstm_fcn.LSTMFCN'>]
<class 'CODE.Train.TS_2_V.main.Classifier_TS2V'>


In [4]:
defence_model = Defence
aug_method = "gaussian_smooth"
defence_model_paramaters = {
    "mother_model": i_model,
    "augmentation": Augmentation.get_method()[aug_method],
    "aug_paramater": dict(),
}
trainer = Trainer(
    dataset="FordB",
    epoch=1000,
    batch_size=128,
    model=defence_model,
    unbais=True,
    device="cuda:0",
    model_P=defence_model_paramaters,
)
trainer.path_parameter = {"aug_method": aug_method}
trainer.__set_output_dir__()
print(trainer.method_path)
trainer.train_and_evaluate(to_device=True, override=False)

/opt/conda/Anaconda3/envs/pytorch_gpu/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


model=Classifier_TS2V_aug_method=gaussian_smooth


In [5]:
trainer.out_dir

'/Project/Project/ADMA2024/v0.1/OUTPUT/train/model=Classifier_TS2V_aug_method=gaussian_smooth/FordB'

In [6]:
def concat_metrics_train_1(mode="train", method="", datasets=None):
    if datasets == None:
        datasets = UNIVARIATE_DATASET_NAMES
    metrics_dfs = []
    for dataset in datasets:
        file_path = os.path.join(
            TRAIN_OUTPUT_PATH, method, dataset, f"{mode}_metrics.csv"
        )

        if os.path.exists(file_path):
            dataset_df = pd.DataFrame([dataset], columns=["dataset"])
            temp_df = pd.read_csv(file_path)
            temp_df = pd.concat([dataset_df] + [temp_df], axis=1)

            metrics_dfs.append(temp_df)
        else:
            print(f"'{file_path}' not found! Skip.")
            return

    final_df = pd.concat(metrics_dfs, ignore_index=False)
    final_df.to_csv(
        os.path.join(
            TRAIN_OUTPUT_PATH,
            f"{mode.upper()}_{'_'.join(method.split(os.path.sep))}_metrics.csv",
        ),
        index=False,
    )

In [7]:
concat_metrics_train_1(mode="train", method=trainer.method_path)
concat_metrics_train_1(mode="test", method=trainer.method_path)

In [18]:
_ = "/Project/Project/ADMA2024/v0.1/OUTPUT/train/model=Classifier_MACNN_aug_method=gaussian_smooth/FordB"
os.path.exists(_)

True